In [19]:
# user defined R installation
import os
os.environ['R_HOME'] = 'D:/Program Files/R-4.5.0' #path to your R installation
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import globalenv
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

from causaloptim_python import run_experiment, extract_prob_dict



# Load R packages
#igraph = importr('igraph')
causaloptim = importr('causaloptim')
base = importr('base')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Data Generation

In [20]:
# Set seed and generate synthetic data in Python
np.random.seed(42)
n = 5000
Z = np.random.binomial(1, 0.5, size=n)
U = np.random.binomial(1, 0.5, size=n)

# X ~ Z + U
logit_X = -1 + 1.5 * Z + 1.2 * U
p_X = 1 / (1 + np.exp(-logit_X))
X = np.random.binomial(1, p_X)

# Y ~ X + U
beta_X, beta_U = 1.0, 1.5
logit_Y = -0.5 + beta_X * X + beta_U * U
p_Y = 1 / (1 + np.exp(-logit_Y))
Y = np.random.binomial(1, p_Y)

df = pd.DataFrame({'Y': Y, 'X': X, 'Z': Z})

# Ground truth ATE
logit_Y1 = -0.5 + beta_X * 1 + beta_U * U
logit_Y0 = -0.5 + beta_X * 0 + beta_U * U
p_Y1 = 1 / (1 + np.exp(-logit_Y1))
p_Y0 = 1 / (1 + np.exp(-logit_Y0))
ATE_true = np.mean(p_Y1 - p_Y0)
print("True ATE:", ATE_true)


True ATE: 0.19877531935424195


#### causaloptim

In [22]:


graph_str = "(Z -+ X, X -+ Y, Ur -+ X, Ur -+ Y)"
leftside = [1, 0, 0, 0]
latent   = [0, 0, 0, 1]
nvals    = [2, 2, 2, 2]
rlconnect = [0, 0, 0, 0]
monotone = [0, 0, 0, 0]

prob_dict = extract_prob_dict(df)


bounds = run_experiment(graph_str, leftside, latent, nvals, rlconnect, monotone, prob_dict)
print("Bounds:", bounds)

Bounds: (<rpy2.robjects.vectors.FloatVector object at 0x000001762E5585D0> [14]
R classes: ('numeric',)
[-0.159200], <rpy2.robjects.vectors.FloatVector object at 0x000001762E559010> [14]
R classes: ('numeric',)
[0.519600])
